## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation - *sur la glace ; ça pourrait permettre d'éviter les faux négatifs dûs à des variations singulier/pluriel quand on essaie d'extraire les termes qui existent déjà dans la taxonomie*)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH présents dans les données

### **Lire le corpus** 

In [103]:
lng = 'fr'
acteur = 'ciusss_centresud'
tag = ''
if tag:
    file = acteur + '/' + acteur + '_' + tag + '.csv'

else:
    if lng == 'fr':
        file = acteur +'.csv'
    if lng == 'en':
        file = acteur + '_en.csv'

In [104]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
from pandas import *
import glob

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist


import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG=lng)


from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

In [105]:
def lire_corpus(acteur = acteur, langue=lng):
    base_path = '../03-corpus/2-data/'
    
    folder_path = path.join(base_path, '1-' + langue, acteur)
    all_files = glob.glob(path.join(folder_path, "*.csv"))
    tags = [f.split('_')[1][:-4] for f in listdir(folder_path)]

    df = DataFrame()
    for f, tag in zip(all_files, tags):
        csv = read_csv(f, encoding='utf-8', sep=',')
        csv = csv[~csv["Address"].str.contains('pdf')] #  Problèmes 
        #csv = csv['text']
        # Using DataFrame.insert() to add a column
        df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Title', 'text']]
        #df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Title', 'Type', 'text']]
    return df

In [106]:
data = lire_corpus(acteur) 
nb_docs = len(data)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 428 documents.


In [107]:
base_path = '../03-corpus/2-data/'
folder_path = path.join(base_path, '1-' + lng, acteur)

data.to_csv(folder_path + '.csv')
data

,Corpus,Sous-corpus,Title,text
0,ciusss_centresud,centr,"Aides à la marche (cannes, béquilles, marchettes)","Déficience physique (sensorielle, langage, aud..."
1,ciusss_centresud,centr,Aides de suppléance à l’audition,"Déficience physique (sensorielle, langage, aud..."
2,ciusss_centresud,centr,Ambulateurs,"Déficience physique (sensorielle, langage, aud..."
3,ciusss_centresud,centr,"Chaussures orthétiques, orthèses plantaires et...","Déficience physique (sensorielle, langage, aud..."
4,ciusss_centresud,centr,Chien d’assistance à la motricité,"Déficience physique (sensorielle, langage, aud..."
...,...,...,...,...
216,ciusss_centresud,centr,Aides techniques - Vidéos,"Déficience physique (sensorielle, langage, aud..."
217,ciusss_centresud,centr,Déroulement d&#039;un avortement,Santé sexuelle Déroulement d'un avortement Ava...
218,ciusss_centresud,centr,physio.hv.ccsmtl@ssss.gouv.qc.ca,"Page introuvable Désolé, il est impossible de ..."
219,ciusss_centresud,centr,COVID-19 : Consultez la section Consultation a...,"Page introuvable Désolé, il est impossible de ..."


### **Nettoyage**

In [108]:
punct = '[!#$%&•►*+,;\/\\<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
# phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

text = [str(t).strip('\n').lower().replace('’', '\'') for t in data['text'].tolist()]
text = [re.sub(spaces, ' ', t) for t in text]
text = [re.sub(postals, ' STOP ', t) for t in text]
text = [re.sub(punct, ' STOP ', t) for t in text]
text = [t.replace("  ", " " ) for t in text]

### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [109]:
def sample_corpus(corpus, ratio):
    n = round(ratio * len(corpus))
    corpus = random.sample(text, n)
    print("On va travailler sur un échantillon correspondant à environ " + str(ratio * 100) + " % des documents du corpus, soit {} documents". format(len(corpus)))
    return " ".join(corpus)
    
corpus = sample_corpus(text, 1)

On va travailler sur un échantillon correspondant à environ 100 % des documents du corpus, soit 428 documents


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [110]:
def filter_mwesw(corpus):
    file_mwesw = '../04-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' STOP ').replace('  ', " ")
    return corpus

In [111]:
corpus = filter_mwesw(corpus)

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [112]:
# Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour voir combien de temps ça devrait 
# prendre au Tree Tagger pour tokeniser et tagger notre corpus
def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))

temps = tok(corpus)

Avec le RegExpTokenizer, notre corpus contient 165710 tokens.
Le POS tagging devrait prendre environ 0 minutes.


In [113]:
def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append([t.split('\t')[0], t.split('\t')[1]])
        except Exception as e:
            output.append(('STOP', 'NAM'))

    return output


In [114]:
tagged = tagging(corpus)
tokens = [t[0] for t in tagged]

KeyboardInterrupt: 

In [ ]:
#tagged

### **Lemmatisation** (FrenchLefffLemmatizer)
Sur pause ; peut-être qu'il vaudrait mieux le faire plus tard (au moment d'identifier si chaque terme existe déjà dans la taxo ou non) afin d'éviter de complexifier toutes les autres étapes

In [ ]:
# from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
# lemmatizer = FrenchLefffLemmatizer()

#### **Mapping POS Tags** (FRMG)
On doit mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)
#http://alpage.inria.fr/frmgwiki/content/tagset-frmg

In [ ]:
# file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

# with open(file_path) as f:
#    csv = read_csv(f)

# treeTag = [term for term in csv['TreeTagger'].tolist()] 
# lefff = [term for term in csv['Lefff'].tolist()]

# mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [ ]:
# df_tagged = DataFrame(tagged, columns=['Token', 'Tag TreeTagger'])
# df_tagged['Tag Lefff'] = df_tagged['Tag TreeTagger'].map(mapping)
# df_tagged['Lemme'] = df_tagged['Token'] # S'il est pas capable de lemmatiser ensuite, on garde l'expression telle quelle

In [ ]:
# tagged_dict = df_tagged.to_dict('records')

In [ ]:
# # tokens = df_tagged['Token'].tolist()
# # tags = df_tagged['Tag Lefff'].tolist()
# # tagged_lefff = [(t, tg) for t,tg in zip(tokens, tags)]

# for term in tagged_dict:
#     if lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff']) == []:
#         term_l = lemmatizer.lemmatize(word=term['Token'])
    
#     elif type(lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])) == str:
#         term_l  = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])

#     else:
#         term_l = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])[0][0]

#     term['Lemme'] = term_l

In [ ]:
# tagged_dict

In [ ]:
# tagged = DataFrame(tagged_dict)
# tagged = tagged.drop(columns=['Tag Lefff']).values.tolist()

# tagged

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [ ]:
def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=1, max_len=10))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

In [ ]:
ngrammes = extr_ngrams(tagged)
#ngrammes_lem = extr_ngrams(tagged_lem)

Avant filtrage, on a 1622035 ngrammes.


In [ ]:
ngrammes

[(['retour', 'NOM'],),
 (['retour', 'NOM'], ['aux', 'PRP:det']),
 (['retour', 'NOM'], ['aux', 'PRP:det'], ['carrières', 'NOM']),
 (['retour', 'NOM'],
  ['aux', 'PRP:det'],
  ['carrières', 'NOM'],
  ['date', 'NOM']),
 (['retour', 'NOM'],
  ['aux', 'PRP:det'],
  ['carrières', 'NOM'],
  ['date', 'NOM'],
  ['de', 'PRP']),
 (['retour', 'NOM'],
  ['aux', 'PRP:det'],
  ['carrières', 'NOM'],
  ['date', 'NOM'],
  ['de', 'PRP'],
  ['publication', 'NOM']),
 (['retour', 'NOM'],
  ['aux', 'PRP:det'],
  ['carrières', 'NOM'],
  ['date', 'NOM'],
  ['de', 'PRP'],
  ['publication', 'NOM'],
  ['5', 'NUM']),
 (['retour', 'NOM'],
  ['aux', 'PRP:det'],
  ['carrières', 'NOM'],
  ['date', 'NOM'],
  ['de', 'PRP'],
  ['publication', 'NOM'],
  ['5', 'NUM'],
  ['août', 'NOM']),
 (['retour', 'NOM'],
  ['aux', 'PRP:det'],
  ['carrières', 'NOM'],
  ['date', 'NOM'],
  ['de', 'PRP'],
  ['publication', 'NOM'],
  ['5', 'NUM'],
  ['août', 'NOM'],
  ['2022', 'NUM']),
 (['retour', 'NOM'],
  ['aux', 'PRP:det'],
  ['carrière

### **Extraction des patrons syntaxiques**

In [ ]:
def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = tuple([t[0] for t in ng])
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

In [ ]:
phrases = extract_patterns(ngrammes)
# phrases_lem = extract_patterns(ngrammes_lem)

In [ ]:
# def freq(phrases):
#     return FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])

# def freq(phrases):
#     return FreqDist([t[0] for t in phrases])

In [ ]:
frequencies = FreqDist(everygrams(tokens, min_len=1, max_len=10))

In [ ]:
frequencies

FreqDist({('de',): 10734, ('STOP',): 9385, ('et',): 5384, ("l'",): 4150, ('.',): 3810, ('des',): 3560, ('les',): 3508, ('la',): 3366, ("d'",): 3234, ('en',): 3148, ...})

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [ ]:
# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

*LONG*

In [ ]:
def filtrer_stopwords(x): # On s'assure aussi que le premier terme du ngramme est un NOM pour avoir des syntagmes nominaux
    if lng == 'en':
        nom = 'NN'
    if lng == 'fr':
        nom = 'NOM'
    return [term for term in x if not 'STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords \
        and not 'NUM' in term[1] and term[1][0] == nom and not '.' in term[0] and not '-' in term[0] and not ':' in term[0]\
        
        # Une parenthèse fermante peut juste se trouver comme dernier token
        # Si une parenthèse est ouverte, elle doit aussi être fermée (et vice versa)
        and not ')' in term[0][:-1] and not ('(' in term[0] and not ')' in term[0]) \
        and not (')' in term[0] and not '(' in term[0])]

In [ ]:
phrases = filtrer_stopwords(phrases)

On retire les n-grammes qui débutent ou se terminent par un token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [ ]:
def filter_len(x):
    return [term for term in x if \
        (len(term[0][0]) > 2 or term[0][0] == '(')  and (len(term[0][-1]) > 2 or term[0][-1] == ')') and \
        len(term[0][0]) < 18 and len(term[0][-1]) < 18]

In [ ]:
phrases = filter_len(phrases)

In [ ]:
phrases

[[('publication',), ['NOM']],
 [('échéance',), ['NOM']],
 [('statut',), ['NOM']],
 [('statut', 'liste'), ['NOM', 'NOM']],
 [('statut', 'liste', 'de', 'rappel'), ['NOM', 'NOM', 'PRP', 'NOM']],
 [('statut', 'liste', 'de', 'rappel', 'salaire'),
  ['NOM', 'NOM', 'PRP', 'NOM', 'NOM']],
 [('liste',), ['NOM']],
 [('liste', 'de', 'rappel'), ['NOM', 'PRP', 'NOM']],
 [('liste', 'de', 'rappel', 'salaire'), ['NOM', 'PRP', 'NOM', 'NOM']],
 [('rappel',), ['NOM']],
 [('rappel', 'salaire'), ['NOM', 'NOM']],
 [('salaire',), ['NOM']],
 [('service',), ['NOM']],
 [('service', 'alimentaire'), ['NOM', 'ADJ']],
 [('service', 'alimentaire', "l'", 'institut'),
  ['NOM', 'ADJ', 'DET:ART', 'NOM']],
 [('service', 'alimentaire', "l'", 'institut', 'national'),
  ['NOM', 'ADJ', 'DET:ART', 'NOM', 'ADJ']],
 [('service',
   'alimentaire',
   "l'",
   'institut',
   'national',
   'de',
   'psychiatrie'),
  ['NOM', 'ADJ', 'DET:ART', 'NOM', 'ADJ', 'PRP', 'NOM']],
 [('service',
   'alimentaire',
   "l'",
   'institut',
  

On retire les n-grammes qui apparaissent moins de 15 fois dans le corpus

In [ ]:
def filter_freq(x):
    return [term for term in x if frequencies[tuple(term[0])] > 15]

In [ ]:
phrases = filter_freq(phrases)

In [ ]:
phrases

[[('publication',), ['NOM']],
 [('échéance',), ['NOM']],
 [('statut',), ['NOM']],
 [('statut', 'liste'), ['NOM', 'NOM']],
 [('statut', 'liste', 'de', 'rappel'), ['NOM', 'NOM', 'PRP', 'NOM']],
 [('statut', 'liste', 'de', 'rappel', 'salaire'),
  ['NOM', 'NOM', 'PRP', 'NOM', 'NOM']],
 [('liste',), ['NOM']],
 [('liste', 'de', 'rappel'), ['NOM', 'PRP', 'NOM']],
 [('liste', 'de', 'rappel', 'salaire'), ['NOM', 'PRP', 'NOM', 'NOM']],
 [('rappel',), ['NOM']],
 [('rappel', 'salaire'), ['NOM', 'NOM']],
 [('salaire',), ['NOM']],
 [('service',), ['NOM']],
 [('service', 'alimentaire'), ['NOM', 'ADJ']],
 [('institut',), ['NOM']],
 [('institut', 'national'), ['NOM', 'ADJ']],
 [('institut', 'national', 'de', 'psychiatrie'), ['NOM', 'ADJ', 'PRP', 'NOM']],
 [('institut', 'national', 'de', 'psychiatrie', 'légale'),
  ['NOM', 'ADJ', 'PRP', 'NOM', 'ADJ']],
 [('institut', 'national', 'de', 'psychiatrie', 'légale', 'philippe-pinel'),
  ['NOM', 'ADJ', 'PRP', 'NOM', 'ADJ', 'ADJ']],
 [('institut',
   'national',

In [ ]:
#phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
phrases = [[term[0], " ".join(term[1])] for term in phrases]
# phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [ ]:
for phrase in phrases:
    phrase.append(frequencies[tuple(phrase[0])])

In [ ]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 71293 occurrences de ngrammes.


In [ ]:
phrases[:15]

[[('publication',), 'NOM', 128],
 [('échéance',), 'NOM', 124],
 [('statut',), 'NOM', 276],
 [('statut', 'liste'), 'NOM NOM', 32],
 [('statut', 'liste', 'de', 'rappel'), 'NOM NOM PRP NOM', 32],
 [('statut', 'liste', 'de', 'rappel', 'salaire'), 'NOM NOM PRP NOM NOM', 16],
 [('liste',), 'NOM', 62],
 [('liste', 'de', 'rappel'), 'NOM PRP NOM', 54],
 [('liste', 'de', 'rappel', 'salaire'), 'NOM PRP NOM NOM', 16],
 [('rappel',), 'NOM', 62],
 [('rappel', 'salaire'), 'NOM NOM', 16],
 [('salaire',), 'NOM', 128],
 [('service',), 'NOM', 176],
 [('service', 'alimentaire'), 'NOM ADJ', 18],
 [('institut',), 'NOM', 1136]]

In [ ]:
DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"])

phrases

[[('publication',), 'NOM', 128],
 [('échéance',), 'NOM', 124],
 [('statut',), 'NOM', 276],
 [('statut', 'liste'), 'NOM NOM', 32],
 [('statut', 'liste', 'de', 'rappel'), 'NOM NOM PRP NOM', 32],
 [('statut', 'liste', 'de', 'rappel', 'salaire'), 'NOM NOM PRP NOM NOM', 16],
 [('liste',), 'NOM', 62],
 [('liste', 'de', 'rappel'), 'NOM PRP NOM', 54],
 [('liste', 'de', 'rappel', 'salaire'), 'NOM PRP NOM NOM', 16],
 [('rappel',), 'NOM', 62],
 [('rappel', 'salaire'), 'NOM NOM', 16],
 [('salaire',), 'NOM', 128],
 [('service',), 'NOM', 176],
 [('service', 'alimentaire'), 'NOM ADJ', 18],
 [('institut',), 'NOM', 1136],
 [('institut', 'national'), 'NOM ADJ', 410],
 [('institut', 'national', 'de', 'psychiatrie'), 'NOM ADJ PRP NOM', 400],
 [('institut', 'national', 'de', 'psychiatrie', 'légale'),
  'NOM ADJ PRP NOM ADJ',
  400],
 [('institut', 'national', 'de', 'psychiatrie', 'légale', 'philippe-pinel'),
  'NOM ADJ PRP NOM ADJ ADJ',
  390],
 [('institut',
   'national',
   'de',
   'psychiatrie',
   'l

In [ ]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(DataFrame(phrases).drop_duplicates())))

Après filtrage, on a 1155 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [ ]:
file_patterns = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist() #[:200] # Pour prendre les 200 structures syntaxiques les plus fréquentes dans les MeSH

In [ ]:
def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NUM' in t[1]] # and not 'NOM NOM' in t[1]
# terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [ ]:
terms = filter_patterns(phrases)
#terms = phrases

In [ ]:
terms

[[('publication',), 'NOM', 128],
 [('échéance',), 'NOM', 124],
 [('statut',), 'NOM', 276],
 [('statut', 'liste'), 'NOM NOM', 32],
 [('statut', 'liste', 'de', 'rappel'), 'NOM NOM PRP NOM', 32],
 [('statut', 'liste', 'de', 'rappel', 'salaire'), 'NOM NOM PRP NOM NOM', 16],
 [('liste',), 'NOM', 62],
 [('liste', 'de', 'rappel'), 'NOM PRP NOM', 54],
 [('liste', 'de', 'rappel', 'salaire'), 'NOM PRP NOM NOM', 16],
 [('rappel',), 'NOM', 62],
 [('rappel', 'salaire'), 'NOM NOM', 16],
 [('salaire',), 'NOM', 128],
 [('service',), 'NOM', 176],
 [('service', 'alimentaire'), 'NOM ADJ', 18],
 [('institut',), 'NOM', 1136],
 [('institut', 'national'), 'NOM ADJ', 410],
 [('institut', 'national', 'de', 'psychiatrie'), 'NOM ADJ PRP NOM', 400],
 [('institut', 'national', 'de', 'psychiatrie', 'légale'),
  'NOM ADJ PRP NOM ADJ',
  400],
 [('institut', 'national', 'de', 'psychiatrie', 'légale', 'philippe-pinel'),
  'NOM ADJ PRP NOM ADJ ADJ',
  390],
 [('psychiatrie',), 'NOM', 952],
 [('psychiatrie', 'légale'), 

In [ ]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 21 % des termes
On avait 71293 ngrammes, on en a maintenant 56376.


In [ ]:
# import pandas as pd

# def extract_terms(liste_terms):
#     file_path = '../04-filtrage/output/'
#     tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
#     tab.sort_values(["Fréquence"], 
#                         axis=0,
#                         ascending=[False], 
#                         inplace=True)

#     return 

#     # file_path = path.join(file_path, tag, tag)                    
#     # tab.to_csv(file_path + '_terms.csv')

# extract_terms(terms)
# #extract_terms(terms_lemmatized)

In [ ]:
for phrase in terms:
    phrase[0] = tuple(phrase[0])


terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
terms_patterns = terms_patterns.to_dict('records')
dict_patterns = {}
for term in terms_patterns:
     exp = term['Expression']
     pattern = term['Structure syntaxique']
     dict_patterns[exp] = pattern

In [ ]:
terms_patterns

[{'Expression': ('publication',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 128},
 {'Expression': ('échéance',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 124},
 {'Expression': ('statut',), 'Structure syntaxique': 'NOM', 'Fréquence': 276},
 {'Expression': ('statut', 'liste'),
  'Structure syntaxique': 'NOM NOM',
  'Fréquence': 32},
 {'Expression': ('statut', 'liste', 'de', 'rappel'),
  'Structure syntaxique': 'NOM NOM PRP NOM',
  'Fréquence': 32},
 {'Expression': ('statut', 'liste', 'de', 'rappel', 'salaire'),
  'Structure syntaxique': 'NOM NOM PRP NOM NOM',
  'Fréquence': 16},
 {'Expression': ('liste',), 'Structure syntaxique': 'NOM', 'Fréquence': 62},
 {'Expression': ('liste', 'de', 'rappel'),
  'Structure syntaxique': 'NOM PRP NOM',
  'Fréquence': 54},
 {'Expression': ('liste', 'de', 'rappel', 'salaire'),
  'Structure syntaxique': 'NOM PRP NOM NOM',
  'Fréquence': 16},
 {'Expression': ('rappel',), 'Structure syntaxique': 'NOM', 'Fréquence': 62},
 {'Expression': ('rappel'

### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [ ]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [ ]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 56376 ngrammes.


In [ ]:
frequencies

FreqDist({('de',): 10734, ('STOP',): 9385, ('et',): 5384, ("l'",): 4150, ('.',): 3810, ('des',): 3560, ('les',): 3508, ('la',): 3366, ("d'",): 3234, ('en',): 3148, ...})

In [ ]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [ ]:
def llr_ngrammes(terms):
    llr = []

    ngrammes = [term[0] for term in terms]
        
    for t in ngrammes:
        if len(t) == 1:
            try:
                llr.append({'Terme' : str(t[0]), 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : fd_tokens[str(t[0])], 'LLR': '-', 'p-value': '-'})
            except Exception as e:
                print(t, str(e))
        else:
            c_prior = frequencies[t[:-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = frequencies[t]             # Le ngramme lui-même P(w1w2w3..wn)

            try:
                res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
                p_value = chi2.sf(res, 1) # 1 degrees of freedom
                #if res == float('-inf') :
                #    res = 50000

                if p_value < 0.01 or (res == float('-inf')):
                    #llr.append({'Collocation' : " ".join(t).replace("' ", "'").replace("( ", "(").replace(" )", ")"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p_value})
                    llr.append({'Terme' : t, 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : c_ngram, 'LLR': res, 'p-value': p_value})
            
            except Exception as e:
                print(t, str(e))
            
    return llr


In [ ]:
significant_coll = llr_ngrammes(terms)
#terms = [{'Collocation' : t[0], 'Structure syntaxique': t[1], 'Fréquence' : t[2]} for t in terms] # Si on veut pas appliquer le LLR

In [ ]:
significant_coll


[{'Terme': 'publication',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 128,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'échéance',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 124,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'statut',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 276,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': ('statut', 'liste'),
  'Structure syntaxique': 'NOM NOM',
  'Fréquence (TF)': 32,
  'LLR': 326.1397517816595,
  'p-value': 6.661459085330385e-73},
 {'Terme': ('statut', 'liste', 'de', 'rappel'),
  'Structure syntaxique': 'NOM NOM PRP NOM',
  'Fréquence (TF)': 32,
  'LLR': 524.0854892981545,
  'p-value': 5.466334642137604e-116},
 {'Terme': ('statut', 'liste', 'de', 'rappel', 'salaire'),
  'Structure syntaxique': 'NOM NOM PRP NOM NOM',
  'Fréquence (TF)': 16,
  'LLR': 186.37541949032467,
  'p-value': 1.9657933771036536e-42},
 {'Terme': 'liste',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 62,
  'LLR': '-',
  'p-value': '-'},
 {'Term

In [ ]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len(terms) - len(significant_coll)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len(terms) - len(significant_coll)) / len(terms) *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 0 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 0 % de nos n-grammes.


In [ ]:
df = DataFrame(significant_coll).sort_values(by = "Fréquence (TF)", ascending=False).drop_duplicates()

# On veut faire join pour tous les termes[collocation]
def join_term(x):
    if type(x) == tuple:
        return " ".join(x).replace("' ", "'").replace("( ", "(").replace(" )", ")")
    else:
        return x

df['Terme'] = df['Terme'].apply(lambda x: join_term(x))

if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 

df.to_csv(output_path)

df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value
29379,institut,NOM,1136,-,-
11970,psychiatrie,NOM,952,-,-
16653,psychiatrie légale,NOM ADJ,832,-inf,1.0
37817,candidature,NOM,570,-,-
43940,philippe-pinel,NOM,466,-,-
...,...,...,...,...,...
2896,applicatifs,NOM,16,-,-
39394,subvention,NOM,16,-,-
30671,directrice de la recherche et de l'enseignemen...,NOM PRP DET:ART NOM KON PRP DET:ART NOM ADJ,16,232.926022,0.0
30670,directrice de la recherche et de l'enseignement,NOM PRP DET:ART NOM KON PRP DET:ART NOM,16,243.719375,0.0


### **Filtrage - Fréquence documentaire**
** Il y aurait quelque chose à modifier ici ; on a une fréquence documentaire de 0 pour les ngrammes qui n'ont plus la forme exacte qu'ils avaient dans le corpus orginal. **

In [ ]:
dfs = {term: len([doc for doc in text if term in doc]) for term in df['Terme'].tolist()}

In [ ]:
max_df = round(0.98 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 98% des documents 
max_df

225

In [ ]:
max_df_observed = dfs[max(dfs, key=dfs.get)] # Voir quelle est la fréquence documentaire maximale qu'on retrouve
max_df_observed

216

In [ ]:
max_df_to_remove = {term:df for term,df in dfs.items() if df > max_df}
max_df_to_remove

{}

In [ ]:
dfs = {term:df for term,df in dfs.items() if df < max_df and df > 0} # Si df = 0, c'est un artefact créé par le prétraitement lui-même

Porter attention aux termes qui sortent ici - en comptant la fréquence documentaire, techniquement on ne devrait pas retrouver de 0

In [ ]:
zeros =  {term:df for term,df in dfs.items() if df == 0}  
print(len(zeros))
zeros 

0


{}

In [ ]:
dfs = DataFrame(list(dfs.items()),columns = ['Terme','Fréquence documentaire (DF)']) 

dfs

,Terme,Fréquence documentaire (DF)
0,institut,216
1,psychiatrie,198
2,psychiatrie légale,188
3,candidature,142
4,philippe-pinel,188
...,...,...
901,applicatifs,8
902,subvention,4
903,directrice de la recherche et de l'enseignemen...,6
904,directrice de la recherche et de l'enseignement,6


In [ ]:
df = df.merge(dfs, on='Terme').drop_duplicates()

df.sort_values(['Fréquence (TF)'], 
            axis=0,
            ascending=[False], 
            inplace=True)

In [ ]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF)
0,institut,NOM,1136,-,-,216
1,psychiatrie,NOM,952,-,-,198
2,psychiatrie légale,NOM ADJ,832,-inf,1.0,188
3,candidature,NOM,570,-,-,142
4,philippe-pinel,NOM,466,-,-,188
...,...,...,...,...,...,...
802,rédaction de devis d'appels d'offres,NOM PRP NOM PRP NOM PRP NOM,16,296.615214,0.0,16
803,gestion des risques,NOM PRP:det NOM,16,188.818022,0.0,16
804,rédaction de devis d'appels,NOM PRP NOM PRP NOM,16,307.151797,0.0,16
805,accès aux installations,NOM PRP:det NOM,16,194.781756,0.0,16


In [ ]:
list_terms = df['Terme'].tolist()

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [ ]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
# kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

# ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'").replace("( ", "(").replace(" )", ")") for ng in ngrammes]

In [ ]:
# ngrammes_kwic = [t for t in ngrammes_kwic if not 'STOP' in t]

In [ ]:
# extrant = DataFrame(columns=['Mot-clé','Concordance', 'Fréquence (TF)'])
# kwic = {w : [] for w in kw} 

In [ ]:
# for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
#     for w in kw:
#         if t.startswith(w):
#             kwic[w].append(t)

In [ ]:
# kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [ ]:
# for term in kw:
#     df_kw = DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence (TF)"])
#     df_kw.sort_values(["Fréquence (TF)"], 
#         axis=0,
#         ascending=[False], 
#         inplace=True)

#     df_kw.insert(0, 'Mot-clé', term)
#     extrant = concat([extrant, df])


# extrant = extrant[extrant['Fréquence (TF)'] > 30] 

# # file_path = '../04-filtrage/output/'
# # file_path = path.join(file_path, acteur, tag)


# # extrant.to_csv(file_path + '_KWIC' +'.csv')

# extrant

### **Extraction de termes MeSH**

In [ ]:
df['isMeSHTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,institut,NOM,1136,-,-,216,False
1,psychiatrie,NOM,952,-,-,198,False
2,psychiatrie légale,NOM ADJ,832,-inf,1.0,188,False
3,candidature,NOM,570,-,-,142,False
4,philippe-pinel,NOM,466,-,-,188,False
...,...,...,...,...,...,...,...
802,rédaction de devis d'appels d'offres,NOM PRP NOM PRP NOM PRP NOM,16,296.615214,0.0,16,False
803,gestion des risques,NOM PRP:det NOM,16,188.818022,0.0,16,False
804,rédaction de devis d'appels,NOM PRP NOM PRP NOM,16,307.151797,0.0,16,False
805,accès aux installations,NOM PRP:det NOM,16,194.781756,0.0,16,False


In [ ]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [ ]:
extr_mesh = tokenizer_mesh.tokenize(list_terms)

In [ ]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,institut,NOM,1136,-,-,216,False
1,psychiatrie,NOM,952,-,-,198,False
2,psychiatrie légale,NOM ADJ,832,-inf,1.0,188,False
3,candidature,NOM,570,-,-,142,False
4,philippe-pinel,NOM,466,-,-,188,False
...,...,...,...,...,...,...,...
802,rédaction de devis d'appels d'offres,NOM PRP NOM PRP NOM PRP NOM,16,296.615214,0.0,16,False
803,gestion des risques,NOM PRP:det NOM,16,188.818022,0.0,16,False
804,rédaction de devis d'appels,NOM PRP NOM PRP NOM,16,307.151797,0.0,16,False
805,accès aux installations,NOM PRP:det NOM,16,194.781756,0.0,16,False


In [ ]:
for t in extr_mesh:
    if t in mesh:
        df.loc[df['Terme'] == t, 'isMeSHTerm'] = 'True'
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,institut,NOM,1136,-,-,216,False
1,psychiatrie,NOM,952,-,-,198,False
2,psychiatrie légale,NOM ADJ,832,-inf,1.0,188,True
3,candidature,NOM,570,-,-,142,False
4,philippe-pinel,NOM,466,-,-,188,False
...,...,...,...,...,...,...,...
802,rédaction de devis d'appels d'offres,NOM PRP NOM PRP NOM PRP NOM,16,296.615214,0.0,16,False
803,gestion des risques,NOM PRP:det NOM,16,188.818022,0.0,16,True
804,rédaction de devis d'appels,NOM PRP NOM PRP NOM,16,307.151797,0.0,16,False
805,accès aux installations,NOM PRP:det NOM,16,194.781756,0.0,16,True


In [ ]:
# Test - ne retenir uniquement que les termes qui sont des MeSH ? 
df[df['isMeSHTerm'] == 'True']

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
2,psychiatrie légale,NOM ADJ,832,-inf,1.0,188,True
53,santé mentale,NOM ADJ,188,-inf,1.0,152,True
68,formation continue,NOM ADJ,162,1559.079655,0.0,150,True
85,qualité de vie,NOM PRP NOM,144,-inf,1.0,140,True
108,curriculum vitae,NOM ADJ,138,-inf,1.0,134,True
134,minorités ethniques,NOM ADJ,132,-inf,1.0,132,True
223,soins infirmiers,NOM NOM,82,905.943524,0.0,30,True
248,congé de maladie,NOM PRP NOM,66,860.098061,0.0,118,True
293,services sociaux,NOM ADJ,54,418.057342,0.0,42,True
342,sécurité civile,NOM ADJ,44,542.392205,0.0,8,True


### **Extraction de termes existant dans la taxonomie**

In [ ]:
df['isTaxoTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
0,institut,NOM,1136,-,-,216,False,False
1,psychiatrie,NOM,952,-,-,198,False,False
2,psychiatrie légale,NOM ADJ,832,-inf,1.0,188,True,False
3,candidature,NOM,570,-,-,142,False,False
4,philippe-pinel,NOM,466,-,-,188,False,False
...,...,...,...,...,...,...,...,...
802,rédaction de devis d'appels d'offres,NOM PRP NOM PRP NOM PRP NOM,16,296.615214,0.0,16,False,False
803,gestion des risques,NOM PRP:det NOM,16,188.818022,0.0,16,True,False
804,rédaction de devis d'appels,NOM PRP NOM PRP NOM,16,307.151797,0.0,16,False,False
805,accès aux installations,NOM PRP:det NOM,16,194.781756,0.0,16,True,False


In [ ]:
file_path = '../04-filtrage/default_taxo_labels.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    default = read_csv(f, sep=';')
    labels = list(dict.fromkeys([str(t).strip().lower() for t in default['Label'].tolist()]))

labels

['1 on 1 support',
 '10 weeks and less ultrasound',
 '12-step facilitation therapy',
 '12-step programs',
 '18f pet bone scan',
 '18f bone pet',
 '18f bone scan',
 '1st generation antipsychotics',
 '2019 ncov testing',
 '2d doppler echocardiography',
 '2d echocardiography',
 '2d ultrasound',
 '2nd generation antipsychotics',
 '3d imaging',
 '3d sonography',
 '3d dental imaging',
 '3d echocardiography',
 '3d fetal ultrasound',
 '3d mammography',
 '3d prenatal ultrasound',
 '3d reconstruction',
 '46, xx disorder of sex development',
 '46,xy disorder of sex development',
 '4d echocardiography',
 '4d prenatal ultrasound',
 'a1c diabetes test',
 'a1c test',
 'abo + rh pnl bld',
 'abo and rh group in cord blood',
 'abpm device',
 'acl reconstruction surgery',
 'acth deficiency',
 'acth stimulation test',
 'adhd assessment',
 'adhd diagnostic',
 'adhd evaluation',
 'adhd testing',
 'adn foetal circulant',
 'aids',
 'aids nephropathy',
 'aids serodiagnosis',
 'aids-associated nephropathy',
 'a

In [ ]:
for t in df['Terme'].tolist():
    if t in labels:
        df.loc[df['Terme'] == t, 'isTaxoTerm'] = 'True'
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
0,institut,NOM,1136,-,-,216,False,False
1,psychiatrie,NOM,952,-,-,198,False,True
2,psychiatrie légale,NOM ADJ,832,-inf,1.0,188,True,False
3,candidature,NOM,570,-,-,142,False,False
4,philippe-pinel,NOM,466,-,-,188,False,False
...,...,...,...,...,...,...,...,...
802,rédaction de devis d'appels d'offres,NOM PRP NOM PRP NOM PRP NOM,16,296.615214,0.0,16,False,False
803,gestion des risques,NOM PRP:det NOM,16,188.818022,0.0,16,True,False
804,rédaction de devis d'appels,NOM PRP NOM PRP NOM,16,307.151797,0.0,16,False,False
805,accès aux installations,NOM PRP:det NOM,16,194.781756,0.0,16,True,False


In [ ]:
# Par exemple, on peut extraire les termes MeSH qui ne sont pas dans la taxonomie, mais qui se trouvent dans le corpus d'intérêt
df[df['isTaxoTerm'] == 'False'][df['isMeSHTerm'] == 'True'] #['Terme'].tolist()

C:\Users\p1115145\AppData\Local\Temp\ipykernel_14116\4006263597.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['isTaxoTerm'] == 'False'][df['isMeSHTerm'] == 'True'] #['Terme'].tolist()


,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
2,psychiatrie légale,NOM ADJ,832,-inf,1.0,188,True,False
68,formation continue,NOM ADJ,162,1559.079655,0.0,150,True,False
85,qualité de vie,NOM PRP NOM,144,-inf,1.0,140,True,False
108,curriculum vitae,NOM ADJ,138,-inf,1.0,134,True,False
134,minorités ethniques,NOM ADJ,132,-inf,1.0,132,True,False
248,congé de maladie,NOM PRP NOM,66,860.098061,0.0,118,True,False
342,sécurité civile,NOM ADJ,44,542.392205,0.0,8,True,False
479,développement durable,NOM ADJ,28,352.79009,0.0,12,True,False
480,résolution de problèmes,NOM PRP NOM,28,455.512684,0.0,24,True,False
470,gestion des ressources humaines,NOM PRP:det NOM ADJ,28,389.090128,0.0,20,True,False


In [115]:
if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 

df['Corpus'] = acteur
df = df[["Corpus", "Terme", "Structure syntaxique",	"Fréquence (TF)", "Fréquence documentaire (DF)", "LLR", "isMeSHTerm", "isTaxoTerm" ]]
df.to_csv(output_path)
df

C:\Users\p1115145\AppData\Local\Temp\ipykernel_14116\1308583101.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Corpus'] = acteur


,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm
0,ciusss_centresud,institut,NOM,1136,216,-,False,False
1,ciusss_centresud,psychiatrie,NOM,952,198,-,False,True
2,ciusss_centresud,psychiatrie légale,NOM ADJ,832,188,-inf,True,False
3,ciusss_centresud,candidature,NOM,570,142,-,False,False
4,ciusss_centresud,philippe-pinel,NOM,466,188,-,False,False
...,...,...,...,...,...,...,...,...
802,ciusss_centresud,rédaction de devis d'appels d'offres,NOM PRP NOM PRP NOM PRP NOM,16,16,296.615214,False,False
803,ciusss_centresud,gestion des risques,NOM PRP:det NOM,16,16,188.818022,True,False
804,ciusss_centresud,rédaction de devis d'appels,NOM PRP NOM PRP NOM,16,16,307.151797,False,False
805,ciusss_centresud,accès aux installations,NOM PRP:det NOM,16,16,194.781756,True,False


### **Add MeSH ID & en/fr correspondance from extracted terms**

In [116]:
df['MeSHID'] = None

In [117]:
df

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,MeSHID
0,ciusss_centresud,institut,NOM,1136,216,-,False,False,None
1,ciusss_centresud,psychiatrie,NOM,952,198,-,False,True,None
2,ciusss_centresud,psychiatrie légale,NOM ADJ,832,188,-inf,True,False,None
3,ciusss_centresud,candidature,NOM,570,142,-,False,False,None
4,ciusss_centresud,philippe-pinel,NOM,466,188,-,False,False,None
...,...,...,...,...,...,...,...,...,...
802,ciusss_centresud,rédaction de devis d'appels d'offres,NOM PRP NOM PRP NOM PRP NOM,16,16,296.615214,False,False,None
803,ciusss_centresud,gestion des risques,NOM PRP:det NOM,16,16,188.818022,True,False,None
804,ciusss_centresud,rédaction de devis d'appels,NOM PRP NOM PRP NOM,16,16,307.151797,False,False,None
805,ciusss_centresud,accès aux installations,NOM PRP:det NOM,16,16,194.781756,True,False,None
